# 🌟 DreamGallery: AI‑Powered Artwork Generator

This notebook implements the **DreamGallery** pipeline:
1. Setup Colab environment (GPU, Drive mount, installs)
2. Download & preprocess WikiArt dataset
3. Define and train a GAN to generate base images
4. Apply neural style transfer to stylize generated art
5. Generate samples, visualize, and save outputs
6. Commit code & results back to GitHub from Colab


## 1. Environment Setup

Enable GPU, mount your Google Drive, and install dependencies.

In [1]:
# 1.1 Check GPU availability
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
# 1.2 Mount Google Drive for persistence
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# 1.3 Install required packages
!pip install --quiet numpy matplotlib tensorflow opencv-python scikit-learn kaggle

## 2. Data Preparation

Download the Painter by Numbers dataset via the Kaggle API and preprocess images.

### 2.1 Configure Kaggle API
Upload your `kaggle.json` credential file under `~/.kaggle/` before running.

In [6]:
# 2.1.1 Create kaggle folder & copy credentials (run only if needed)
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

### 2.2 Download & extract dataset

In [7]:
# 2.2.1 Download The Met Open Access dataset via Kaggle CLI
!kaggle datasets download \
    -d metmuseum/the-metropolitan-museum-of-art-open-access \
    -p data/raw

# 2.2.2 Unzip all contents into data/raw/
!unzip -q data/raw/the-metropolitan-museum-of-art-open-access.zip \
    -d data/raw


Dataset URL: https://www.kaggle.com/datasets/metmuseum/the-metropolitan-museum-of-art-open-access
License(s): CC0-1.0


In [ ]:
# 2.2.2 Unzip all contents into data/raw/
!unzip -q data/raw/the-metropolitan-museum-of-art-open-access.zip \
    -d data/raw


replace data/raw/MetObjects.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [4]:
!ls /content/data/raw

ls: cannot access '/content/data/raw': No such file or directory


In [21]:
import os
import time
import pandas as pd
import requests
from tqdm import tqdm

# Paths
CSV_PATH = 'data/raw/MetObjects.csv'
IMG_DIR  = 'data/raw/images'
os.makedirs(IMG_DIR, exist_ok=True)

# 1. Load metadata, suppressing the dtype warning
meta = pd.read_csv(CSV_PATH, low_memory=False)

# 2. Filter to CC0 paintings
meta = meta[meta['Is Public Domain'] == True]
meta = meta[meta['Medium'].str.contains('paint', case=False, na=False)]

# 3. Helper: fetch image URL from The Met API
API_URL = 'https://collectionapi.metmuseum.org/public/collection/v1/objects/{}'
def fetch_image_url(object_id):
    res = requests.get(API_URL.format(object_id), timeout=5)
    if not res.ok:
        return None
    data = res.json()
    # Prefer smaller, web‑sized image for speed
    return data.get('primaryImageSmall') or data.get('primaryImage')

# 4. Download loop
failed = []
for object_id in tqdm(meta['Object ID'].unique(), total=meta['Object ID'].nunique()):
    out_path = os.path.join(IMG_DIR, f"{object_id}.jpg")
    if os.path.exists(out_path):
        continue  # already downloaded

    url = fetch_image_url(object_id)
    if not url:
        failed.append(object_id)
        continue

    try:
        resp = requests.get(url, timeout=10)
        resp.raise_for_status()
        with open(out_path, 'wb') as f:
            f.write(resp.content)
    except Exception as e:
        failed.append(object_id)
    time.sleep(0.01)  # throttle to avoid rate‑limit

# 5. Summary
print(f"Downloaded {len(os.listdir(IMG_DIR))} images, {len(failed)} failures.")


100%|██████████| 8048/8048 [56:23<00:00,  2.38it/s]

Downloaded 7365 images, 683 failures.


### 2.3 Preprocess images (resize + normalize)
Create `data/processed/128x128/` with `.npy` arrays for fast loading.

In [9]:
import os, cv2, numpy as np

RAW_DIR = 'data/raw/train'
PROC_DIR = 'data/processed/128x128'
os.makedirs(PROC_DIR, exist_ok=True)

def preprocess_and_save(src_dir, dst_dir, size=(128,128)):
    for fname in os.listdir(src_dir):
        path = os.path.join(src_dir, fname)
        img = cv2.imread(path)
        if img is None: continue
        img = cv2.resize(img, size)
        img = img.astype('float32') / 255.0
        np.save(os.path.join(dst_dir, fname.split('.')[0] + '.npy'), img)

preprocess_and_save(RAW_DIR, PROC_DIR)

FileNotFoundError: [Errno 2] No such file or directory: 'data/raw/train'

In [ ]:
# Quick check of processed data shape
arr = np.load(os.path.join(PROC_DIR, os.listdir(PROC_DIR)[0]))
print('Sample shape:', arr.shape)

## 3. GAN Model Definition
Define generator, discriminator, and training loop.

In [23]:
import tensorflow as tf
from tensorflow.keras import layers, Model

LATENT_DIM = 100
IMG_SHAPE = (128,128,3)

def build_generator(latent_dim=LATENT_DIM):
    inp = layers.Input(shape=(latent_dim,))
    x = layers.Dense(16*16*128, activation='relu')(inp)
    x = layers.Reshape((16,16,128))(x)
    x = layers.UpSampling2D()(x)
    x = layers.Conv2D(128,3,padding='same', activation='relu')(x)
    x = layers.UpSampling2D()(x)
    x = layers.Conv2D(64,3,padding='same', activation='relu')(x)
    out = layers.Conv2D(3,3,padding='same', activation='tanh')(x)
    return Model(inp, out, name='Generator')

def build_discriminator(img_shape=IMG_SHAPE):
    inp = layers.Input(shape=img_shape)
    x = layers.Conv2D(64,3,strides=2,padding='same')(inp)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Conv2D(128,3,strides=2,padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Flatten()(x)
    out = layers.Dense(1, activation='sigmoid')(x)
    return Model(inp, out, name='Discriminator')

generator = build_generator()
discriminator = build_discriminator()
generator.summary()


Model: "Generator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32768)          │     3,309,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d (UpSampling2D)    │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 32, 32, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_1 (UpSampling2D)  │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 64, 64)     │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 64, 64, 3)      │         1,731 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,532,675 (13.48 MB)

 Trainable params: 3,532,675 (13.48 MB)

 Non-trainable params: 0 (0.00 B)

## 4. GAN Training Loop
Load processed images, compile models, and train the GAN.

In [ ]:
import glob
ndef load_processed(dir_path):
    files = glob.glob(f'{dir_path}/*.npy')
    data = [np.load(fp) for fp in files]
    return np.array(data)

# Load data
images = load_processed(PROC_DIR)
print('Dataset size:', images.shape)

# Labels for real/fake
real_labels = np.ones((images.shape[0], 1))
fake_labels = np.zeros((images.shape[0], 1))

# Compile discriminator
discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Build combined model
discriminator.trainable = False
z = layers.Input(shape=(LATENT_DIM,))
img = generator(z)
validity = discriminator(img)
combined = Model(z, validity)
combined.compile(optimizer='adam', loss='binary_crossentropy')

# Training parameters
EPOCHS = 5000
BATCH = 64

for epoch in range(1, EPOCHS+1):
    idx = np.random.randint(0, images.shape[0], BATCH)
    real_imgs = images[idx]
    noise = np.random.normal(0, 1, (BATCH, LATENT_DIM))
    gen_imgs = generator.predict(noise)

    # Train on real and fake
    d_loss_real = discriminator.train_on_batch(real_imgs, np.ones((BATCH,1)))
    d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((BATCH,1)))
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train generator
    g_loss = combined.train_on_batch(noise, np.ones((BATCH,1)))

    # Log every 500 epochs
    if epoch % 500 == 0:
        print(f"Epoch {epoch} / {EPOCHS}  [D loss: {d_loss[0]:.4f}, acc.: {100*d_loss[1]:.2f}%]  [G loss: {g_loss:.4f}]")

    # Save model checkpoints
    if epoch % 1000 == 0:
        generator.save(f'results/gan_generator_epoch{epoch}.h5')


## 5. Neural Style Transfer
Use TensorFlow's pretrained VGG19 to apply style transfer to GAN outputs.

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import vgg19
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
from tensorflow.keras.models import Model

def load_and_process(path, target_size=(128,128)):
    img = tf.io.read_file(path)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.resize(img, target_size)
    img = img[tf.newaxis, ...]
    return vgg19.preprocess_input(img*255.0)

def deprocess(x):
    x = x.reshape((x.shape[1], x.shape[2], 3))
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    x = x[:, :, ::-1]
    return tf.clip_by_value(x, 0, 255) / 255.0

# Load VGG19 for style and content
vgg = vgg19.VGG19(weights='imagenet', include_top=False)
style_layers = ['block1_conv1','block2_conv1','block3_conv1','block4_conv1']
content_layers = ['block5_conv2']
outputs = [vgg.get_layer(name).output for name in (style_layers + content_layers)]
style_model = Model(vgg.input, outputs)

def gram_matrix(tensor):
    channels = int(tensor.shape[-1])
    a = tf.reshape(tensor, [-1, channels])
    return tf.matmul(a, a, transpose_a=True)

def compute_loss(comb, content, style):
    comb_feats = style_model(comb)
    style_feats = style_model(style)
    content_feats = style_model(content)

    # Content loss
    c_loss = tf.reduce_mean((comb_feats[-1] - content_feats[-1])**2)

    # Style loss
    s_loss = 0
    weight_per_style = 1.0 / len(style_layers)
    for cf, sf in zip(comb_feats[:len(style_layers)], style_feats[:len(style_layers)]):
        s_loss += weight_per_style * tf.reduce_mean((gram_matrix(cf) - gram_matrix(sf))**2)

    return c_loss + 1e-2 * s_loss

optimizer = tf.optimizers.Adam(learning_rate=5.0)

def style_transfer(content_path, style_path, iterations=200):
    content = load_and_process(content_path)
    style = load_and_process(style_path)
    comb = tf.Variable(content)

    for i in range(iterations):
        with tf.GradientTape() as tape:
            loss = compute_loss(comb, content, style)
        grads = tape.gradient(loss, comb)
        optimizer.apply_gradients([(grads, comb)])

    return deprocess(comb.numpy())  # final stylized image

## 6. Inference & Visualization
Generate new images with the trained GAN, apply style transfer, and display.

In [ ]:
import matplotlib.pyplot as plt

# 6.1 Generate base art
noise = np.random.normal(size=(1, LATENT_DIM))
gen_img = generator.predict(noise)
gen_img = (gen_img[0] * 0.5) + 0.5  # rescale from [-1,1] to [0,1]

# Save temporarily
cv2.imwrite('results/base_art.jpg', (gen_img*255).astype('uint8')[...,::-1])

# 6.2 Apply style transfer
stylized = style_transfer('results/base_art.jpg', 'path/to/your/style.jpg', iterations=100)

# 6.3 Display results
fig, axes = plt.subplots(1,2, figsize=(10,5))
axes[0].imshow(gen_img); axes[0].set_title('GAN Output'); axes[0].axis('off')
axes[1].imshow(stylized); axes[1].set_title('Stylized Output'); axes[1].axis('off')


## 7. Save Models & Results to Drive
Persist your trained generator and final outputs to Google Drive.

In [ ]:
# 7.1 Save model
generator.save('/content/drive/MyDrive/dreamgallery/results/gan_generator_final.h5')

# 7.2 Save stylized image
import imageio
imageio.imwrite('/content/drive/MyDrive/dreamgallery/results/stylized_final.jpg', (stylized*255).astype('uint8'))

## 8. Commit & Push to GitHub
Use Colab’s shell to push your updated code and results back to your GitHub repo.

In [ ]:
# Configure Git (first-time only)
!git config --global user.name 'Your Name'
!git config --global user.email 'you@example.com'

# Stage, commit, and push
!git add .
!git commit -m 'Add trained GAN model and generated artwork'
!git push https://<YOUR_TOKEN>@github.com/yourusername/dreamgallery.git HEAD:main

---
**Congrats!** You’ve run the entire DreamGallery pipeline in Colab—from raw data to stylized masterpieces—
and saved everything to both Google Drive and GitHub. Feel free to fork, modify, and share!